In [1]:
# 여기어때 국내숙소 (서울지역) 수집을 목표로 합니다.
# url : https://www.yeogi.com/domestic-accommodations?keyword=서울&page=1

In [ ]:
# 추가 패키지 설치
# !pip install supabase # 수파베이스 SDK 설치
# !pip install selenium # 헤드리스 브라우저를 위한 테스트 자동화 툴
# !pip install beautifulsoup4 # html 파싱 툴
# !pip install tqdm # 프로그래스 표시 

In [2]:
import supabase # 수파베이스 SDK
import urllib.parse # url 파싱
import re # 정규 표현식
from tqdm import tqdm
from dotenv import load_dotenv # 환경변수 로드
import os
from glob import glob
import platform

#### 헤드리스 브라우저 준비
___

In [ ]:
!pip install webdriver-manager
!pip install selenium
!pip install beautifulsoup4
!pip install tqdm
!pip install supabase

In [4]:
# 헤드리스 브라우저 드라이버 객체 생성
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
service = Service(executable_path=driver_path)

def get_driver():
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--blink-settings=imagesEnabled=false')
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    return driver

driver = get_driver()

#### 스크랩 실행
___

In [5]:
# 1차 크롤링 : 숙소 상품 페이지 url을 수집
base_url = 'https://www.yeogi.com'
urls = [f'{base_url}/domestic-accommodations?keyword=서울&page={i}' for i in range(1, 58)]
links = []


In [6]:
for index in tqdm(range(0, len(urls))):
    driver.get(urls[index])
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    page_links_and_imgs = [
        base_url + urllib.parse.urlparse(link['href']).path
        for link in soup.find_all('a', href=re.compile(r'/domestic-accommodations/\d+'))
    ]
    links.extend(page_links_and_imgs)


100%|██████████| 57/57 [00:33<00:00,  1.69it/s]


In [7]:
# 브라우저 드라이버 종료
driver.close()
driver.quit()

#### DB 저장
___

In [8]:
# 환경 변수 로드
load_dotenv()
SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_ANON_KEY = os.getenv("SUPABASE_ANON_KEY")
# 수파베이스 접속
sb = supabase.create_client(SUPABASE_URL, SUPABASE_ANON_KEY)

In [9]:
# 중복확인
len(links), len(set(links))

(1140, 1140)

In [10]:
# 저장
sb.table('links').insert([{'url': url} for url in links]).execute()

APIResponse[TypeVar](data=[{'id': 4, 'created_at': '2024-05-03T02:53:30.842592+00:00', 'url': 'https://www.yeogi.com/domestic-accommodations/70163'}, {'id': 5, 'created_at': '2024-05-03T02:53:30.842592+00:00', 'url': 'https://www.yeogi.com/domestic-accommodations/1742'}, {'id': 6, 'created_at': '2024-05-03T02:53:30.842592+00:00', 'url': 'https://www.yeogi.com/domestic-accommodations/779'}, {'id': 7, 'created_at': '2024-05-03T02:53:30.842592+00:00', 'url': 'https://www.yeogi.com/domestic-accommodations/844'}, {'id': 8, 'created_at': '2024-05-03T02:53:30.842592+00:00', 'url': 'https://www.yeogi.com/domestic-accommodations/895'}, {'id': 9, 'created_at': '2024-05-03T02:53:30.842592+00:00', 'url': 'https://www.yeogi.com/domestic-accommodations/65523'}, {'id': 10, 'created_at': '2024-05-03T02:53:30.842592+00:00', 'url': 'https://www.yeogi.com/domestic-accommodations/49461'}, {'id': 11, 'created_at': '2024-05-03T02:53:30.842592+00:00', 'url': 'https://www.yeogi.com/domestic-accommodations/177

In [11]:
# 확인
import pandas as pd
pd.DataFrame(sb.table('links').select('url').execute().data)


,url
0,https://www.yeogi.com/domestic-accommodations/...
1,https://www.yeogi.com/domestic-accommodations/...
2,https://www.yeogi.com/domestic-accommodations/779
3,https://www.yeogi.com/domestic-accommodations/844
4,https://www.yeogi.com/domestic-accommodations/895
...,...
995,https://www.yeogi.com/domestic-accommodations/...
996,https://www.yeogi.com/domestic-accommodations/...
997,https://www.yeogi.com/domestic-accommodations/...
998,https://www.yeogi.com/domestic-accommodations/...


#### 2차 크롤링 - 숙소 상세페이지별 정보 스크랩
___
* 4배속으로 수집을 하기 위해 cralwer.py 자동 스크립트를 제공함.
* 멀티 프로세싱으로 동시에 여러개의 브라우저로 수집하는 방식.
* 과다하게 사용할 경우, 사이트 성능 저하를 일으키거나, 수집이 막힐수 있다.
* 실행방법 : 터미널 창 > lecture08/ 폴더로 이동 > python crawler.py 4 입력 (4는 프로세스 수, 변경가능)

In [ ]:
# 한개 페이지만 스크랩 해보겠다.
driver = webdriver.Chrome(service=service, options=chrome_options)
driver.get(links[0])
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
title = soup.select_one("title").getText() if soup.select_one("title") else ""
description = soup.select_one('meta[name="description"]')['content'] if soup.select_one('meta[name="description"]') else ""
canonical_url = soup.select_one('link[rel="canonical"]')['href'] if soup.select_one('link[rel="canonical"]') else ""
address = soup.select_one('.css-1mwesgd').getText() if soup.select_one('.css-1mwesgd') else ""
id = urllib.parse.urlparse(canonical_url).path.split('/')[-1]
image = soup.select_one('section[aria-label="갤러리"] img')['src'] if soup.select_one('section[aria-label="갤러리"] img') else ""

In [47]:
# 드라이버 종료
driver.close()
driver.quit()

In [40]:
# 결과 확인
from IPython.display import display, HTML
display(HTML(f"""
<style>
.title {{
    width: 40px;
}}
div {{
    width: 500px;
}}
td {{
    text-align: left;
}}
</style>
<div>
<table>
<tr><td class="title">이미지</td><td><img src='{image}' width='300' height='300'></td></tr>
<tr><td class="title">제목</td><td>{title}</td></tr>
<tr><td class="title">설명</td><td>{description}</td></tr>
<tr><td class="title">주소</td><td>{address}</td></tr>
<tr><td class="title">URL</td><td>{canonical_url}</td></tr>
</table>
</div>
"""))


이미지,
제목,"서울신라호텔, 을지로·시청·명동, 서울 - 여기어때 특가"
설명,"전통이라는 지붕 위에 모더니즘적 디자인 요소를 가미, 삶에 여유와 품격을 한층 높여 주는 프리미엄 라이프스타일 공간으로 변화를 거듭해 오는 세계 최고의 럭셔리 호텔입니다"
주소,서울 중구 장충동2가 202
URL,https://www.yeogi.com/domestic-accommodations/70163


In [41]:
# DB 저장
sb.table('scraps').insert([{'id': id, 'title': title, 'description': description, 'url': canonical_url, 'address': address, 'image': image}]).execute()


APIResponse[TypeVar](data=[{'id': 70163, 'created_at': '2024-04-27T02:41:42.61151+00:00', 'title': '서울신라호텔, 을지로·시청·명동, 서울 - 여기어때 특가', 'description': '전통이라는 지붕 위에 모더니즘적 디자인 요소를 가미, 삶에 여유와 품격을 한층 높여 주는 프리미엄 라이프스타일 공간으로 변화를 거듭해 오는 세계 최고의 럭셔리 호텔입니다', 'url': 'https://www.yeogi.com/domestic-accommodations/70163', 'address': '서울 중구 장충동2가 202', 'image': 'https://www.yeogi.com/_next/image?url=https%3A%2F%2Fimage.goodchoice.kr%2Fexhibition%2FitemContents%2F444f1e5a26087b15ef64e6089397d748.jpg&w=1200&q=75'}], count=None)

In [44]:
# 데이터 확인 
pd.DataFrame(sb.table('scraps').select('*').execute().data).set_index('id')

,created_at,title,description,url,address,image
id,,,,,,
70163,2024-04-27T02:41:42.61151+00:00,"서울신라호텔, 을지로·시청·명동, 서울 - 여기어때 특가","전통이라는 지붕 위에 모더니즘적 디자인 요소를 가미, 삶에 여유와 품격을 한층 높여...",https://www.yeogi.com/domestic-accommodations/...,서울 중구 장충동2가 202,https://www.yeogi.com/_next/image?url=https%3A...


In [46]:
# DB 초기화 (자동 스크랩 스크립트로 다시 스크랩 예정)
sb.table('scraps').delete().eq('id', 70163).execute()

APIResponse[TypeVar](data=[{'id': 70163, 'created_at': '2024-04-27T02:41:42.61151+00:00', 'title': '서울신라호텔, 을지로·시청·명동, 서울 - 여기어때 특가', 'description': '전통이라는 지붕 위에 모더니즘적 디자인 요소를 가미, 삶에 여유와 품격을 한층 높여 주는 프리미엄 라이프스타일 공간으로 변화를 거듭해 오는 세계 최고의 럭셔리 호텔입니다', 'url': 'https://www.yeogi.com/domestic-accommodations/70163', 'address': '서울 중구 장충동2가 202', 'image': 'https://www.yeogi.com/_next/image?url=https%3A%2F%2Fimage.goodchoice.kr%2Fexhibition%2FitemContents%2F444f1e5a26087b15ef64e6089397d748.jpg&w=1200&q=75'}], count=None)

____
#### 1차 크롤링 끝
____

In [59]:
[row['url'] for row in sb.table('links').select('url').execute().data]



['https://www.yeogi.com/domestic-accommodations/70163',
 'https://www.yeogi.com/domestic-accommodations/1742',
 'https://www.yeogi.com/domestic-accommodations/65523',
 'https://www.yeogi.com/domestic-accommodations/895',
 'https://www.yeogi.com/domestic-accommodations/844',
 'https://www.yeogi.com/domestic-accommodations/779',
 'https://www.yeogi.com/domestic-accommodations/49461',
 'https://www.yeogi.com/domestic-accommodations/72748',
 'https://www.yeogi.com/domestic-accommodations/73221',
 'https://www.yeogi.com/domestic-accommodations/1777',
 'https://www.yeogi.com/domestic-accommodations/50263',
 'https://www.yeogi.com/domestic-accommodations/6798',
 'https://www.yeogi.com/domestic-accommodations/5666',
 'https://www.yeogi.com/domestic-accommodations/6534',
 'https://www.yeogi.com/domestic-accommodations/4060',
 'https://www.yeogi.com/domestic-accommodations/5803',
 'https://www.yeogi.com/domestic-accommodations/4988',
 'https://www.yeogi.com/domestic-accommodations/63624',
 'http